This notebook calculates all of the SNeCO CDFs, 
some stats about the fraction of CO emission at 150 pc and 
writes the SNeCDFs to a fits file and writes 6 versions of the allPixel CDFs to a fits file. 

'../Data/3.2a.SNeCDFs.fits'

SNeCDFs = {'Detections','NonDetections','DetectionsII','NonDetectionsII','DetectionsIa','NonDetectionsIa'  : 'DetectionsIbc','NonDetectionsIbc','DetectionsUnC','NonDetectionsUnC'}

each of these rows holds CDFdetection,ydetection,CDFdetection_mgsd_ca,ydetection_mgsd_ca,CDFdetection_mgsd_mda,ydetection_mgsd_mda

'../Data/3.2a_CDFAllPixelsInts.fits'           :   CDFAllPixelsInts, yAllPixelsInts
'../Data/3.2a_CDFAllPixelsLims.fits'           :   CDFAllPixelsLims, yAllPixelsLims
'../Data/3.2a_CDFAllPixelsInts_ca.fits'        :   CDFAllPixelsInts_ca, yAllPixelsInts_ca
'../Data/3.2a_CDFAllPixelsLims_ca.fits'        :   CDFAllPixelsLims_ca, yAllPixelsLims_ca
'../Data/3.2a_CDFAllPixelsInts_mgsd_mda.fits'  :   CDFAllPixelsInts_mgsd_mda, yAllPixelsInts_mgsd_mda
'../Data/3.2a_CDFAllPixelsLims_mgsd_mda.fits'  :   CDFAllPixelsLims_mgsd_mda, yAllPixelsLims_mgsd_mda


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from astropy import table
from astropy.table import Table, join
from astropy.io import ascii
import astropy.io.fits as pyfits

import os

import sys
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')
#sys.path.append('/Users/nessmayker/Desktop/NessPythonFunctions')
from detectionFraction import detecFrac
from cumulativeDistFunc import makeCDF
from findPercentiles import findStats
from flattenList import flatten
from nonZeroError import findErrVals

#import SNe data 

SNe_Gal_File = '../Data/3.SNe+GalData.csv'
SNe_Gal_Data = Table.read(SNe_Gal_File, format='csv') 

intenFile = '../Data/3.IntenData.csv'
intenData = Table.read(intenFile, format='csv')

database  = join(SNe_Gal_Data, intenData, keys=('SN_name', 'galaxy'))


In [4]:
database

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile,PACovFile,IFRFile,MixedMaps,IFRResizedMaps,MassMaps,NatRes,NatInt,pc60Int,pc90Int,pc120Int,pc150Int,pc500Int,pc750Int,pc1000Int,NatErr,pc60Err,pc90Err,pc120Err,pc150Err,pc500Err,pc750Err,pc1000Err,NatEW,pc60EW,pc90EW,pc120EW,pc150EW,pc500EW,pc750EW,pc1000EW,NatSNR,pc60SNR,pc90SNR,pc120SNR,pc150SNR,pc500SNR,pc750SNR,pc1000SNR,alphaCO,pc60mgsd,pc90mgsd,pc120mgsd,pc150mgsd,pc500mgsd,pc750mgsd,pc1000mgsd,pc60mgsdErr,pc90mgsdErr,pc120mgsdErr,pc150mgsdErr,pc500mgsdErr,pc750mgsdErr,pc1000mgsdErr
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str74,str112,str97,str37,str54,str35,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ngc1566,ASASSN-14ha,II,65.0059,-54.9381,258.9038295074317,423.1062029210927,68.05383,1.8849889,12.188041,0.0003484111476313,65.00159,-54.93801,17.69,214.7,29.5,216.83093,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1566/ngc1566_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1566/ngc1566_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC1566_alphaCO21_Sun+20.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1566/ngc1566_12m+7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/ngc1566_irac1_gauss7p5.fits,../Data/MixedMaps/ngc1566_Mixed.fits,../Data/ReprojectedInfraredMaps/ngc1566_reshaped.fits,../Data/MassMaps/ngc1566_Mass.fits,107.5715044627487,68.0538330078125,nan,nan,65.95396423339844,61.926334381103516,27.371265411376953,26.06928253173828,27.377416610717773,1.8849889039993286,nan,nan,1.6742128133773804,1.3591481447219849,0.5764862298965454,0.3906446397304535,0.27018043398857117,12.188040733337402,nan,nan,12.932862281799316,13.762829780578613,19.730430603027344,29.85703468322754,43.81895446777344,36.10304170142571,nan,nan,39.39401473122755,45.56260818335635,47.47947824580116,66.73400804814882,101.33012300911419,4.304510126195559,nan,nan,283.8995069054033,266.5625334216323,117.81988913005836,112.21549064052044,117.84636702990916,nan,nan,7.20666600858929,5.850466951955691,2.481490814201981,1.6815338074637531,1.1629944140037154
ngc4321,SN1901B,I,185.6971,15.8238,545.5920792633563,276.5735561756638,5.0824146,1.0344207,2.544713,0.000462934650464,185.72887,15.822304,15.21,156.2,38.5,182.86105,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc4321/ngc4321_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc4321/ngc4321_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC4321_alphaCO21_Sun+20.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc4321/ngc4321_12m+7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/ngc4321_irac1_gauss7p5.fits,../Data/MixedMaps/ngc4321_Mixed.fits,../Data/ReprojectedInfraredMaps/ngc4321_reshaped.fits,../Data/MassMaps/ngc4321_Mass.fits,122.89275219530901,5.6628522872924805,nan,nan,5.6628522872924805,5.4790143966674805,6.016371250152588,5.822849273681641,5.013950347900391,1.029056191444397,nan,nan,1.029056191444397,0.8251489996910095,0.23007987439632416,0.13967318832874298,0.08780597150325775,2.744281053543091,nan,nan,2.744281053543091,4.029609203338623,10.151252746582031,13.131563186645508,13.50499439239502,5.502957306290559,nan,n

In [5]:
# Find which SNe have nan values and which have finite
nans   = np.argwhere(np.isnan(database["pc150Int"]))
finite = np.argwhere(np.isfinite(database["pc150Int"]))

numSNe    = len(database["pc150Int"])
numFinite = len(finite)
numNans   = len(nans)

# Create array of intensity, errpr, & SNR values where measurements are finite
SNRvalues    = database["pc150SNR"][finite]
values       = database["pc150Int"][finite]
errors       = database["pc150Err"][finite]
alphaCO_Sun  = database["alphaCO"][finite]

# Find which are considered non-detections and which are detections
detections_idx, um    = np.where(SNRvalues >= 3.0)
nondetections_idx, um = np.where(SNRvalues < 3.0)

detections_int    = flatten(values[detections_idx])

nondetections_int = flatten(values[nondetections_idx])

numDetect    = len(detections_int)
numNonDetect = len(nondetections_int)

# assign nondetections upper limits as a value of 3x the noise.
nondetections_ul = flatten(3 * errors[nondetections_idx])

print("Out of", numSNe, "supernovae,", numFinite, "or", round(numFinite/numSNe*100, 0), "percent of the sample are within the PHANGS-ALMA footprint at 150pc resolution.")
print("Out of these,", numDetect, "or", round(numDetect/numFinite*100, 0), "percent of the sample have a SNR of 3.0 or greater and are classified as coinciding in areas with detectable CO2-1 emission.")
print("The remaining", numNonDetect, "are assigned an upper limit CO2-1 intensity measurement of 3 times the noise value at their location.")


Out of 66 supernovae, 62 or 94.0 percent of the sample are within the PHANGS-ALMA footprint at 150pc resolution.
Out of these, 37 or 60.0 percent of the sample have a SNR of 3.0 or greater and are classified as coinciding in areas with detectable CO2-1 emission.
The remaining 25 are assigned an upper limit CO2-1 intensity measurement of 3 times the noise value at their location.


In [7]:
for i in range(len(database["SN_type"][finite])):
    print(database["SN_type"][finite][i])

['II']
['I']
['II']
['II']
['unclassified']
['I']
['unclassified']
['unclassified']
['I']
['Ia']
['II']
['II']
['II']
['II']
['II']
['II']
['II']
['Ia']
['Ib']
['Ic']
['II']
['II']
['Ia']
['II']
['II']
['II']
['Ia']
['II']
['Ia']
['II']
['II']
['II']
['II']
['II']
['II']
['II']
['Ic']
['Ic']
['Ia']
['Ia']
['II']
['II']
['II']
['unclassified']
['II']
['II']
['Ia']
['II']
['II']
['Ia']
['Ia']
['II']
['Ic']
['Ia']
['II']
['II']
['Ib']
['II']
['II']
['II']
['Ia']
['Ic']


In [4]:
# Sort measurements by SN Type and separate detections from nondetections.

SNtypes = database["SN_type"][finite]

typeII_idx, um    = np.where(SNtypes=="II")
typeIa_idx, um    = np.where(SNtypes=="Ia")
typeIb_idx, um    = np.where(SNtypes=="Ib")
typeIc_idx, um    = np.where(SNtypes=="Ic")
typeIbIc_idx, um   = np.where(SNtypes=="Ib/c")
typeIbc_idx = np.concatenate((typeIb_idx, typeIc_idx, typeIbIc_idx), axis = 0)
typeIbc_idx.sort()  
typeI_idx, um = np.where(SNtypes=="I")
typeUnk_idx, um = np.where(SNtypes=="unclassified")
typeUnC_idx = np.concatenate((typeI_idx, typeUnk_idx), axis = 0)
typeUnC_idx.sort()

numTypeII  = len(typeII_idx)
numTypeIa  = len(typeIa_idx)
numTypeIbc = len(typeIbc_idx)
numTypeUnC = len(typeUnC_idx)

# find indicies of each SN type where there is both a detection and nondetection
detectionsII_idx = np.intersect1d(typeII_idx, detections_idx)
nondetectionsII_idx = np.intersect1d(typeII_idx, nondetections_idx)
numtypeIId  = len(detectionsII_idx)
numtypeIInd = len(nondetectionsII_idx)
detectionsII_int = flatten(values[detectionsII_idx])
nondetectionsII_ul = flatten(3 * errors[nondetectionsII_idx])

detectionsIa_idx = np.intersect1d(typeIa_idx, detections_idx)
nondetectionsIa_idx = np.intersect1d(typeIa_idx, nondetections_idx)
numtypeIad  = len(detectionsIa_idx)
numtypeIand = len(nondetectionsIa_idx)
detectionsIa_int = flatten(values[detectionsIa_idx])
nondetectionsIa_ul = flatten(3 * errors[nondetectionsIa_idx])

detectionsIbc_idx = np.intersect1d(typeIbc_idx, detections_idx)
nondetectionsIbc_idx = np.intersect1d(typeIbc_idx, nondetections_idx)
numtypeIbcd  = len(detectionsIbc_idx)
numtypeIbcnd = len(nondetectionsIbc_idx)
detectionsIbc_int = flatten(values[detectionsIbc_idx])
nondetectionsIbc_ul = flatten(3 * errors[nondetectionsIbc_idx])

detectionsUnC_idx = np.intersect1d(typeUnC_idx, detections_idx)
nondetectionsUnC_idx = np.intersect1d(typeUnC_idx, nondetections_idx)
numtypeUnCd  = len(detectionsUnC_idx)
numtypeUnCnd = len(nondetectionsUnC_idx)
detectionsUnC_int = flatten(values[detectionsUnC_idx])
nondetectionsUnC_ul = flatten(3 * errors[nondetectionsUnC_idx])


# This is how to access these indexed lists
# print(database["SN_type"][finite][typeIIdetections_idx])

print("We have ", round(numtypeIId/numTypeII * 100,0), "percent CO 2-1 detections at the location of our",  numTypeII, "typeII SNe.")
print("We have ", round(numtypeIad/numTypeIa * 100,0), "percent CO 2-1 detections at the location of our",  numTypeIa, "typeIa SNe.")
print("We have ", round(numtypeIbcd/numTypeIbc * 100,0), "percent CO 2-1 detections at the location of our",  numTypeIbc, "typeIb/c SNe.")
print("We have ", round(numtypeUnCd/numTypeUnC * 100,0), "percent CO 2-1 detections at the location of our",  numTypeUnC, "untyped SNe.")


We have  56.0 percent CO 2-1 detections at the location of our 36 typeII SNe.
We have  33.0 percent CO 2-1 detections at the location of our 12 typeIa SNe.
We have  86.0 percent CO 2-1 detections at the location of our 7 typeIb/c SNe.
We have  100.0 percent CO 2-1 detections at the location of our 7 untyped SNe.


In [5]:
# Make CDFs of detections and by each type.

frac = numNonDetect/numFinite
fracII = numtypeIInd/numTypeII
fracIa = numtypeIand/numTypeIa
fracIbc = numtypeIbcnd/numTypeIbc
fracUnC = numtypeUnCnd/numTypeUnC

def yStart(numSample):
    start = 1.0/numSample
    return(start)

startAll = yStart(numFinite)       
CDFdetection, ydetection       = makeCDF(detections_int, min=frac+startAll)
CDFnondetection, ynondetection = makeCDF(nondetections_ul, min = startAll, max=frac)

startII = yStart(numTypeII)      
CDFdetectionII, ydetectionII      = makeCDF(detectionsII_int, min=fracII+startII)
CDFnondetectionII, ynondetectionII = makeCDF(nondetectionsII_ul, min = startII, max=fracII)

startIa = yStart(numTypeIa)       
CDFdetectionIa, ydetectionIa      = makeCDF(detectionsIa_int, min=fracIa+startIa)
CDFnondetectionIa, ynondetectionIa = makeCDF(nondetectionsIa_ul, min = startIa, max=fracIa)

startIbc = yStart(numTypeIbc)       
CDFdetectionIbc, ydetectionIbc      = makeCDF(detectionsIbc_int, min=fracIbc+startIbc)
CDFnondetectionIbc, ynondetectionIbc = makeCDF(nondetectionsIbc_ul, min = startIbc, max=fracIbc)

startUnC = yStart(numTypeUnC)       
CDFdetectionUnC, ydetectionUnC      = makeCDF(detectionsUnC_int, min=fracUnC+startUnC)
CDFnondetectionUnC, ynondetectionUnC = makeCDF(nondetectionsUnC_ul, min = startUnC, max=fracUnC)


In [6]:
# Convert to molecular gas surface density using constant alphaCO

alphaCO = 4.35/0.65

detections_mgsd_ca       = [x * alphaCO for x in detections_int]
nondetections_mgsd_ca    = [x * alphaCO for x in nondetections_ul]

detectionsII_mgsd_ca     = [x * alphaCO for x in detectionsII_int]
nondetectionsII_mgsd_ca  = [x * alphaCO for x in nondetectionsII_ul]

detectionsIa_mgsd_ca     = [x * alphaCO for x in detectionsIa_int]
nondetectionsIa_mgsd_ca  = [x * alphaCO for x in nondetectionsIa_ul]

detectionsIbc_mgsd_ca    = [x * alphaCO for x in detectionsIbc_int]
nondetectionsIbc_mgsd_ca = [x * alphaCO for x in nondetectionsIbc_ul]

detectionsUnC_mgsd_ca    = [x * alphaCO for x in detectionsUnC_int]
nondetectionsUnC_mgsd_ca = [x * alphaCO for x in nondetectionsUnC_ul]

CDFdetection_mgsd_ca, ydetection_mgsd_ca            = makeCDF(detections_mgsd_ca, min=frac+startAll)
CDFnondetection_mgsd_ca, ynondetection_mgsd_ca      = makeCDF(nondetections_mgsd_ca, min = startAll, max=frac)

CDFdetectionII_mgsd_ca, ydetectionII_mgsd_ca        = makeCDF(detectionsII_mgsd_ca, min=fracII+startII)
CDFnondetectionII_mgsd_ca, ynondetectionII_mgsd_ca   = makeCDF(nondetectionsII_mgsd_ca, min = startII, max=fracII)

CDFdetectionIa_mgsd_ca, ydetectionIa_mgsd_ca        = makeCDF(detectionsIa_mgsd_ca, min=fracIa+startIa)
CDFnondetectionIa_mgsd_ca, ynondetectionIa_mgsd_ca   = makeCDF(nondetectionsIa_mgsd_ca, min = startIa, max=fracIa)

CDFdetectionIbc_mgsd_ca, ydetectionIbc_mgsd_ca      = makeCDF(detectionsIbc_mgsd_ca, min=fracIbc+startIbc)
CDFnondetectionIbc_mgsd_ca, ynondetectionIbc_mgsd_ca = makeCDF(nondetectionsIbc_mgsd_ca, min = startIbc, max=fracIbc)

CDFdetectionUnC_mgsd_ca, ydetectionUnC_mgsd_ca      = makeCDF(detectionsUnC_mgsd_ca, min=fracUnC+startUnC)
CDFnondetectionUnC_mgsd_ca, ynondetectionUnC_mgsd_ca = makeCDF(nondetectionsUnC_mgsd_ca, min = startUnC, max=fracUnC)



In [7]:
# Convert to molecular gas surface density using Sun+20 alphaCO


detections_mgsd_mda       = [a*b for a,b in zip(detections_int, flatten(alphaCO_Sun[detections_idx]))]
nondetections_mgsd_mda    = [a*b for a,b in zip(nondetections_ul, flatten(alphaCO_Sun[nondetections_idx]))]

detectionsII_mgsd_mda     = [a*b for a,b in zip(detectionsII_int, flatten(alphaCO_Sun[detectionsII_idx]))]
nondetectionsII_mgsd_mda  = [a*b for a,b in zip(nondetectionsII_ul, flatten(alphaCO_Sun[nondetectionsII_idx]))]

detectionsIa_mgsd_mda     = [a*b for a,b in zip(detectionsIa_int, flatten(alphaCO_Sun[detectionsIa_idx]))]
nondetectionsIa_mgsd_mda  = [a*b for a,b in zip(nondetectionsIa_ul, flatten(alphaCO_Sun[nondetectionsIa_idx]))]

detectionsIbc_mgsd_mda    = [a*b for a,b in zip(detectionsIbc_int, flatten(alphaCO_Sun[detectionsIbc_idx]))]
nondetectionsIbc_mgsd_mda = [a*b for a,b in zip(nondetectionsIbc_ul, flatten(alphaCO_Sun[nondetectionsIbc_idx]))]

detectionsUnC_mgsd_mda    = [a*b for a,b in zip(detectionsUnC_int, flatten(alphaCO_Sun[detectionsUnC_idx]))]
nondetectionsUnC_mgsd_mda = [a*b for a,b in zip(nondetectionsUnC_ul, flatten(alphaCO_Sun[nondetectionsUnC_idx]))]

CDFdetection_mgsd_mda, ydetection_mgsd_mda       = makeCDF(detections_mgsd_mda, min=frac+startAll)
CDFnondetection_mgsd_mda, ynondetection_mgsd_mda = makeCDF(nondetections_mgsd_mda, min = startAll, max=frac)

CDFdetectionII_mgsd_mda, ydetectionII_mgsd_mda      = makeCDF(detectionsII_mgsd_mda, min=fracII+startII)
CDFnondetectionII_mgsd_mda, ynondetectionII_mgsd_mda = makeCDF(nondetectionsII_mgsd_mda, min = startII, max=fracII)

CDFdetectionIa_mgsd_mda, ydetectionIa_mgsd_mda      = makeCDF(detectionsIa_mgsd_mda, min=fracIa+startIa)
CDFnondetectionIa_mgsd_mda, ynondetectionIa_mgsd_mda = makeCDF(nondetectionsIa_mgsd_mda, min = startIa, max=fracIa)

CDFdetectionIbc_mgsd_mda, ydetectionIbc_mgsd_mda      = makeCDF(detectionsIbc_mgsd_mda, min=fracIbc+startIbc)
CDFnondetectionIbc_mgsd_mda, ynondetectionIbc_mgsd_mda = makeCDF(nondetectionsIbc_mgsd_mda, min = startIbc, max=fracIbc)

CDFdetectionUnC_mgsd_mda, ydetectionUnC_mgsd_mda      = makeCDF(detectionsUnC_mgsd_mda, min=fracUnC+startUnC)
CDFnondetectionUnC_mgsd_mda, ynondetectionUnC_mgsd_mda = makeCDF(nondetectionsUnC_mgsd_mda, min = startUnC, max=fracUnC)



In [8]:

SNeCDFs = Table({'Detections'      : [CDFdetection,ydetection,CDFdetection_mgsd_ca,ydetection_mgsd_ca,CDFdetection_mgsd_mda,ydetection_mgsd_mda],
                'NonDetections'    : [CDFnondetection,ynondetection,CDFnondetection_mgsd_ca,ynondetection_mgsd_ca,CDFnondetection_mgsd_mda,ynondetection_mgsd_mda],
                'DetectionsII'     : [CDFdetectionII,ydetectionII,CDFdetectionII_mgsd_ca,ydetectionII_mgsd_ca,CDFdetectionII_mgsd_mda,ydetectionII_mgsd_mda],
                'NonDetectionsII'  : [CDFnondetectionII,ynondetectionII,CDFnondetectionII_mgsd_ca,ynondetectionII_mgsd_ca,CDFnondetectionII_mgsd_mda,ynondetectionII_mgsd_mda],
                'DetectionsIa'     : [CDFdetectionIa,ydetectionIa,CDFdetectionIa_mgsd_ca,ydetectionIa_mgsd_ca,CDFdetectionIa_mgsd_mda,ydetectionIa_mgsd_mda],
                'NonDetectionsIa'  : [CDFnondetectionIa,ynondetectionIa,CDFnondetectionIa_mgsd_ca,ynondetectionIa_mgsd_ca,CDFnondetectionIa_mgsd_mda,ynondetectionIa_mgsd_mda],
                'DetectionsIbc'    : [CDFdetectionIbc,ydetectionIbc,CDFdetectionIbc_mgsd_ca,ydetectionIbc_mgsd_ca,CDFdetectionIbc_mgsd_mda,ydetectionIbc_mgsd_mda],
                'NonDetectionsIbc' : [CDFnondetectionIbc,ynondetectionIbc,CDFnondetectionIbc_mgsd_ca,ynondetectionIbc_mgsd_ca,CDFnondetectionIbc_mgsd_mda,ynondetectionIbc_mgsd_mda] ,
                'DetectionsUnC'    : [CDFdetectionUnC,ydetectionUnC,CDFdetectionUnC_mgsd_ca,ydetectionUnC_mgsd_ca,CDFdetectionUnC_mgsd_mda,ydetectionUnC_mgsd_mda],
                'NonDetectionsUnC' : [CDFnondetectionUnC,ynondetectionUnC,CDFnondetectionUnC_mgsd_ca,ynondetectionUnC_mgsd_ca,CDFnondetectionUnC_mgsd_mda,ynondetectionUnC_mgsd_mda]})


SNeCDFs.write('../Data/3.2a.SNeCDFs.fits', overwrite=True)



In [9]:
#!pip install reproject
from reproject import reproject_interp
########### GETTING ALL PIXEL VALUES HERE #################

# filename: /data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc2997/ngc2997_12m+7m+tp_co21_150pc_broad_mom0.fits

# Isolate unique galaxy names
unique_by_name = table.unique(database, keys='galaxy')
intfile, errfile, acofile = [],[],[]

# Create list of Filenames
for i in range(len(unique_by_name)):
    prefix = "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/"
    galaxy = unique_by_name['galaxy'][i]
    telOrient = unique_by_name['telOrient'][i]
    int_suffix = "_co21_150pc_broad_mom0.fits"
    err_suffix = "_co21_150pc_broad_emom0.fits"
    intname =  prefix + galaxy + "/" + galaxy + "_" + telOrient + int_suffix
    errname =  prefix + galaxy + "/" + galaxy + "_" + telOrient + err_suffix
    
    GalName = galaxy.strip().upper()
    
    if GalName == 'CIRCINUS':
        GalName = 'ESO097-013'
        
    aconame =  "/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/" + GalName + '_alphaCO21_Sun+20.fits'
    
    intfile.append(intname)
    errfile.append(errname)
    acofile.append(aconame)    
    
# Pull intensity values out of galaxy maps for CDF and assign them as three sigma measurements or below

threeSigmaInts, nonThreeSigmaInts, threeSigma_mgsd_mda, nonThreeSigma_mgsd_mda = [],[],[],[]

for i in range(len(intfile)):
    
    if os.path.isfile(intfile[i]):
        
        hdu_int  = pyfits.open(intfile[i])
        hdu_err  = pyfits.open(errfile[i])
      
        intmap      = hdu_int[0].data 
        errmap      = hdu_err[0].data 
        
        f_int    = intmap.flatten()
        f_err    = errmap.flatten()
        
        keep     = np.where(np.isfinite(f_int))
        inten    = f_int[keep]
        error    = f_err[keep]
        
        lowErr = findErrVals(error)
        
        for j in range(len(error)):
            
            if error[j] <= lowErr:
                error[j] == lowErr
            
            elif np.isnan(error[j]):
                error[j] == lowErr
            
            else:
                pass
            
        if os.path.isfile(acofile[i]):
            
            hdu_aco  = pyfits.open(acofile[i])
            
            acomap, footprint = reproject_interp(hdu_aco, hdu_int[0].header) 

            f_aco    = acomap.flatten()
            alphaCO  = f_aco[keep]
                    
            for j in range(len(alphaCO)):

                if np.isnan(alphaCO[j]):
                    alphaCO[j] = 6.7

                else:
                    pass
                
        else: alphaCO = np.full(len(inten),6.7,dtype=float)
            
        for j in range(len(inten)):
            
            if inten[j] < 0.0:
                nonThreeSigmaInts.append(0.0)
                nonThreeSigma_mgsd_mda.append(0.0)
            
            elif inten[j]/error[j] <= 3.0:
                nonThreeSigmaInts.append(3 * error[j])
                nonThreeSigma_mgsd_mda.append(3 * error[j] * alphaCO[j])
            
            else:
                threeSigmaInts.append(inten[j])
                threeSigma_mgsd_mda.append(inten[j] * alphaCO[j])
    
    else:
        print("No 150pc file for", unique_by_name['galaxy'][i])


        

No 150pc file for ngc1068
No 150pc file for ngc1672
No 150pc file for ngc4579


In [10]:

# Make CDFs of all Pixels

sortedInts          = np.sort(threeSigmaInts)
sortedLims          = np.sort(nonThreeSigmaInts)
sortedInts_mgsd_mda = np.sort(threeSigma_mgsd_mda)
sortedLims_mgsd_mda = np.sort(nonThreeSigma_mgsd_mda)

numInts = len(sortedInts)
numLims = len(sortedLims)
numAll  = numLims+numInts

frac = numLims/(numAll)

start = yStart(numAll)

CDFAllPixelsInts, yAllPixelsInts = makeCDF(sortedInts, min=frac+start)
CDFAllPixelsLims, yAllPixelsLims = makeCDF(sortedLims, min=start, max = frac)

sortedInts_ca = sortedInts * 6.7
sortedLims_ca = sortedLims * 6.7

CDFAllPixelsInts_ca, yAllPixelsInts_ca = makeCDF(sortedInts_ca, min=frac+start)
CDFAllPixelsLims_ca, yAllPixelsLims_ca = makeCDF(sortedLims_ca, min=start, max = frac)

CDFAllPixelsInts_mgsd_mda, yAllPixelsInts_mgsd_mda = makeCDF(sortedInts_mgsd_mda, min=frac+start)
CDFAllPixelsLims_mgsd_mda, yAllPixelsLims_mgsd_mda = makeCDF(sortedLims_mgsd_mda, min=start, max = frac)


In [11]:
CDFAllPixelsInts = Table({"CDFAllPixelsInts":[CDFAllPixelsInts, yAllPixelsInts]})
CDFAllPixelsInts.write('../Data/3.2a_CDFAllPixelsInts.fits', overwrite = True)

CDFAllPixelsLims = Table({"CDFAllPixelsLims":[CDFAllPixelsLims, yAllPixelsLims]})
CDFAllPixelsLims.write('../Data/3.2a_CDFAllPixelsLims.fits', overwrite = True)

CDFAllPixelsInts_ca = Table({"CDFAllPixelsInts_ca":[CDFAllPixelsInts_ca, yAllPixelsInts_ca]})
CDFAllPixelsInts_ca.write('../Data/3.2a_CDFAllPixelsInts_ca.fits', overwrite = True)

CDFAllPixelsLims_ca = Table({"CDFAllPixelsLims_ca":[CDFAllPixelsLims_ca, yAllPixelsLims_ca]})
CDFAllPixelsLims_ca.write('../Data/3.2a_CDFAllPixelsLims_ca.fits', overwrite = True)

CDFAllPixelsInts_mgsd_mda = Table({"CDFAllPixelsInts_mgsd_mda":[CDFAllPixelsInts_mgsd_mda, yAllPixelsInts_mgsd_mda]})
CDFAllPixelsInts_mgsd_mda.write('../Data/3.2a_CDFAllPixelsInts_mgsd_mda.fits', overwrite = True)

CDFAllPixelsLims_mgsd_mda = Table({"CDFAllPixelsLims_mgsd_mda":[CDFAllPixelsLims_mgsd_mda, yAllPixelsLims_mgsd_mda]})
CDFAllPixelsLims_mgsd_mda.write('../Data/3.2a_CDFAllPixelsLims_mgsd_mda.fits', overwrite = True)